In [2]:
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose


## optain open dataset in torchvision

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/Users/danfer/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459044803/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## create dataloaders


In [5]:
batch_size = 64
training_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("shape of X[N, C, H, W]: ", X.shape)
    print("shape of y: ", y.shape, y.dtype)
    break


shape of X[N, C, H, W]:  torch.Size([64, 1, 28, 28])
shape of y:  torch.Size([64]) torch.int64


## create model


In [6]:
# detect gpu
device = "cuda" if torch.cuda.is_available() else "cpu"
print("using {} device".format(device))

# define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## optimize parameters


In [53]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

def train(dataloader, model, loss, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        prediction = model(X)
        loss_v = loss(prediction, y)

        optimizer.zero_grad()
        loss_v.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss_v, current = loss_v.item(), batch * len(X)
            print("loss: " + str(loss_v) + "[" + str(current) + "/" + str(size) + "]")

def test(dataloader, model, loss):
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            prediction = model(X)
            test_loss += loss(prediction, y).item()
            correct += (prediction.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batch
    correct /= size
    print("test error: " + "accuracy: " + str(100*correct) + "%, average loss: " + str(test_loss))

## fit the model

In [54]:
epoch = 5
for iter in range(epoch):
    print(f"epoch {iter+1}")
    train(training_dataloader, model, loss, optimizer)
    test(test_dataloader, model, loss)
print("done!")

epoch 1
loss: 0.5318375825881958[0/60000]
loss: 0.6534724831581116[6400/60000]
loss: 0.44896790385246277[12800/60000]
loss: 0.6760584712028503[19200/60000]
loss: 0.6053077578544617[25600/60000]
loss: 0.58930504322052[32000/60000]
loss: 0.6338049173355103[38400/60000]
loss: 0.6772085428237915[44800/60000]
loss: 0.6575143933296204[51200/60000]
loss: 0.6044848561286926[57600/60000]
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.8484e+00,
         0.0000e+00, 8.9554e+00, 5.1553e+00, 9.8205e+00],
        [8.8946e+00, 3.0287e+00, 1.5421e+01, 6.9472e+00, 1.1694e+01, 1.1903e+00,
         1.3447e+01, 0.0000e+00, 6.9157e+00, 0.0000e+00],
        [4.5705e+00, 1.5665e+01, 3.9194e+00, 7.4185e+00, 7.5874e+00, 0.0000e+00,
         2.5621e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.9976e+00, 1.1961e+01, 2.2077e+00, 6.1679e+00, 4.9305e+00, 0.0000e+00,
         1.2864e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [7.0053e+00, 1.9291e+00, 7.2476e+00, 5.2071e+00, 4.911

KeyboardInterrupt: 

## save the model

In [35]:
torch.save(model.state_dict(), "quick_start_model.pth")
print("saved pytorch model")


saved pytorch model


## load the model


In [36]:
model_loaded = NeuralNetwork()
model_loaded.load_state_dict(torch.load("quick_start_model.pth"))


<All keys matched successfully>

## make predictions

In [37]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"]

x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    prediction_m = model(x)
    prediction, actual = classes[prediction_m[0].argmax(0)], classes[y]
    print("predict: " + prediction + ", actual: " + actual)



predict: Ankle boot, actual: Ankle boot
